<a href="https://colab.research.google.com/github/jonasvdbran/Masterthesis-SCRPPP/blob/main/Look_ahead_Housekeeping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we import the container_classes file. **Upload container_classes.py via the next block of code.**

In [2]:
from google.colab import files
uploaded = files.upload()

Saving container_classes.py to container_classes.py


In [3]:
from container_classes import *

We again will test the algorithm on our example bay.

In [4]:
module=Module([Bay([Stack([Container(1),Container(2)],4),
    Stack([Container(3),Container(3),Container(4),Container(1)],4),Stack([Container(2)],4),Stack([Container(1),Container(4),Container(4),Container(5)],4)],4)],4)
print(module)

Bay 0:
  | 1 |   | 5
  | 4 |   | 4
2 | 3 |   | 4
1 | 3 | 2 | 1


# Ideal Stacks

We want to find the ideal stacks in the module to place a container with a certain time slot in. Ideal stacks are defined in section 4.2 and are stacks where a container can be placed in, without needing to be housekept again.

In [5]:
def IdealStacks(module, value,alpha,index):
    ideal_stacks = []
    for i,bay in enumerate(module):
      for j, stack in enumerate(bay):
        # Empty stacks are always ideal
        if stack.is_empty() and (i,j)!=index:
          ideal_stacks.append(((i,j), stack,0))

        elif stack.is_full():
          continue
        elif min(stack).value>= value and IsAcceptable(stack) and (i,j)!=index:
            valid = True
            blocking_costs=0
            for k, container in enumerate(stack):
                same_below = [below for below in stack[:k] if below == container.value]
                n = len(same_below)+1
                # Calculate blocking probability
                p = 1 - 1 / n
                blocking_costs+=p
            same_timeslot=[container for container in stack if container.value==value]
            n=len(same_timeslot)+1
            p=1-1/n
            blocking_costs+=p
            # If the blocking costs exceed alpha, we will housekeep -> stack not ideal
            if blocking_costs>alpha:
              valid=False
            if valid:
                ideal_stacks.append(((i,j), stack,p))
    return ideal_stacks

With \alpha=0.6, we have that for the container with time slot 2 in the first stack, stack 3 is ideal. It's blocking probability in this stack becomes 0.5, as there is already 1 container in the stack.

In [6]:
IdealStacks(module,2,0.6,(0,0))

[((0, 2), Stack([Container(2, '', False)]), 0.5)]

# Blocking containers

The next function determines the blocking containers in the module.

In [8]:
def blocking_containers(module,alpha):
  blockings=[]
  for i,bay in enumerate(module):
    for j,stack in enumerate(bay):
      for k,container in enumerate(stack):
        if any(stack[-k].value > below for below in stack[:-k]):
          # If there is a contain with higher time slot than the ones beneath,
          #add this container together with the containers above it to the blocking containers.
          blockings.extend((i, j, l) for l in range(len(stack)-k, len(stack)))
        elif any(stack[-k].value == below for below in stack[:-k]):
          count = sum(stack[-k].value == below for below in stack[:-k])
          p=1-1/(count+1)
          # If a container has a blocking probability higher than alpha, it will be housekept.
          if p>alpha:
            blockings.extend((i, j, l) for l in range(len(stack)-k, len(stack)))
  return sorted(list(set(blockings)))

In [9]:
blocking_containers(module,0.6)

[(0, 0, 1), (0, 1, 2), (0, 1, 3), (0, 3, 1), (0, 3, 2), (0, 3, 3)]

# Housekeeping costs

We implement the estimated housekeeping costs as in equation (4.1): HC(c)=\alpha(a_c+1)+\alpha \min_{\bar{s} \in \bar{s}\setminus \{\bar{s}(c)\}}M_{\bar{s}}+h(c). The function CalculateHousekeepingCosts also implements algorithm 7 via EstimateBadHousekeepings.

In [14]:
# Finds the containers in M_{\bar{s}}
def get_to_move(container_values: list,threshold: int)-> np.array:
  smallest_under=np.minimum.accumulate(np.array(container_values))
  # Find containers that need to be moved when placing a container w value threshold in stack (i.e. the smallest value under it is smaller than threshold)
  return np.array(container_values)[smallest_under<threshold]

In [10]:
def CalculateHousekeepingCosts(module,alpha):
  h = {}
  for x in range(len(module)):
    for y in range(len(module[x])):
      for z in range(len(module[x][y])):
        h[(x,y,z)]=0

  H = {}
  for x in range(len(module)):
    for y in range(len(module[x])):
      for z in range(len(module[x][y])):
        H[(x,y,z)]=0
  # Algorithm 7
  def EstimateBadHousekeepings():
    blockings=blocking_containers(module,alpha)
    for c_b in blockings:
      # Add cost of alpha as we need to move it.
      H[c_b]+=alpha
      containers_above=module[c_b[0]][c_b[1]][c_b[2]+1:]
      stack_index=(c_b[0],c_b[1])
      # Check containers above it
      for c in containers_above:
        # Add cost of alpha: need to move containers above.
        H[c_b]+=alpha
        ideal_stacks=IdealStacks(module,c,alpha,stack_index)
        # If it has no ideal stack, add another cost of alpha.
        if not(ideal_stacks):
          h[c_b]+= alpha

      value=module[c_b].value
      ideal_stacks=IdealStacks(module,value,alpha,stack_index)
      # If the container has no ideal stack to move to:
      if not(ideal_stacks):

        min_M_s=min(len(get_to_move([container.value for container in stack.containers],value)) for i,bay in enumerate(module) for j,stack in enumerate(bay) if not (i,j)==c_b[:2] and not stack.is_full())
        destination_stacks=[((i,j),stack) for i,bay in enumerate(module) for j,stack in enumerate(bay) if len(get_to_move([container.value for container in stack.containers],value))==min_M_s and not stack.is_full()and not (i,j)==c_b[:2]]
        destination_stack=random.choice(destination_stacks)

        M_s=list(get_to_move([container.value for container in destination_stack[1].containers],value))
        # Check for each container in M_s if they have an ideal stack to move to.
        for c in M_s:

          H[c_b]+=alpha
          ideal_stacks=[stack for stack in IdealStacks(module,c,alpha,destination_stack[0]) if stack[0]!=stack_index]
          if not(ideal_stacks):
            h[c_b]+= alpha
      else:
         h[c_b]+=alpha*min([ideal_stack[2] for ideal_stack in ideal_stacks])
  EstimateBadHousekeepings()
  for pos in H:
    if h[pos]:
      H[pos]+=h[pos]
  return H,h


In [15]:
HC,h=CalculateHousekeepingCosts(module,0.6)

In [17]:
HC

{(0, 0, 0): 0,
 (0, 0, 1): 0.8999999999999999,
 (0, 1, 0): 0,
 (0, 1, 1): 0,
 (0, 1, 2): 2.4,
 (0, 1, 3): 0.6,
 (0, 2, 0): 0,
 (0, 3, 0): 0,
 (0, 3, 1): 4.199999999999999,
 (0, 3, 2): 3.0,
 (0, 3, 3): 1.7999999999999998}

# Look-ahead housekeeping

In this section, we'll introduce the Look-ahead houseeping algorithm, algorithm 8. First we introduce a helper function that finds the destination stack as described in the algorithm.

In [22]:
def get_destination_stack(module,container_value,alpha,stack_index,excluded_stacks):

  ideal_stacks = IdealStacks(module,container_value,alpha,stack_index)
  print(f"Ideal stacks: {ideal_stacks}")
  if ideal_stacks and not all(pos in excluded_stacks for pos, _,_ in ideal_stacks):
    # Pick one with minimum extra blocking costs
    min_val = min(x[2] for x in ideal_stacks if x[0] not in excluded_stacks)
    min_stacks = [x for x in ideal_stacks if x[2] == min_val]
    # From these choose the stack in which the minimal time slot is lowest
    min_time_slot = min(min(stack[1]).value if not stack[1].is_empty() else -1 for stack in min_stacks)
    potential_stacks = [stack for stack in min_stacks if stack[1].is_empty() or min(stack[1]).value == min_time_slot]
    # If there are still multiple, choose one randomly
    destination_stack_index=random.choice(potential_stacks)[0]
  # Else place in a stack with lowest minimum time slot
  else:
    possible_stacks= [(i,j) for i,bay in enumerate(module) for j,stack in enumerate(bay) if (i,j) not in excluded_stacks and not(stack.is_full())]
    destination_stack_index=min(possible_stacks,key=lambda x: min(module[x[0]][x[1]]).value,default=[])
  return destination_stack_index

In [28]:
def LookAheadHousekeeping(module,alpha):
  HC,_=CalculateHousekeepingCosts(module,alpha)
  SC,_=CalculateShiftingCosts(module)

  # Consider the containers for which the housekeeping costs are cheaper than the shifting costs
  housekeepable_containers = [pos for pos in HC if HC[pos] < SC[pos] and HC[pos]>0]
  module=module.copy()

  blockings=blocking_containers(module,alpha)
  moved=False
  moves=[]

  while housekeepable_containers:

    print(f"housekeepable containers:{housekeepable_containers}")
    # 1. Determine which container to housekeep
    # Determine positions for which HC is minimal and HC<SC
    min_HC_positions = [pos for pos in housekeepable_containers if HC[pos] == min(HC[p] for p in housekeepable_containers)]
    # Determine positions for which the container in the module has maximal time slot, HC is minimal and HC<SC
    max_positions = [pos for pos in min_HC_positions if module[pos] == max(module[pos] for pos in min_HC_positions)]
    # Pick one randomly if there are still multiple containers possible.
    container_pos = random.choice(max_positions)
    print(f"Housekeeping:{container_pos}")
    print(f"HC:{HC[container_pos]}")
    print(f"SC:{SC[container_pos]}")
    source_stack=(container_pos[0],container_pos[1])
    container_value=module[container_pos].value

    # 2. Determine to which stack to move
    new_module=module.copy()
    new_moves=[]
    ideal_stacks = IdealStacks(new_module,container_value,alpha,source_stack)
    if ideal_stacks:
      # Pick the ones for which the extra shifting costs p are minimal (same container stacks)
      min_val = min(stack[2] for stack in ideal_stacks)
      min_stacks = [stack for stack in ideal_stacks if stack[2] == min_val]
      # From these pick the one with lowest minimum time slot
      min_time_slot = min(min(stack[1]).value if not stack[1].is_empty() else -1 for stack in min_stacks)
      potential_stacks = [stack for stack in min_stacks if stack[1].is_empty() or min(stack[1]).value == min_time_slot]

      # If there are still multiple, choose one randomly
      destination_stack_index=random.choice(potential_stacks)[0] # If there are multiple with height being maximum, pick one randomly

    else:
      # If no ideal stack, choose a destination stack in which we will place the container correctly
      min_M_S=min(len(get_to_move([container.value for container in stack.containers],container_value)) for i,bay in enumerate(new_module) for j,stack in enumerate(bay) if not stack.is_full() and not (i,j)==source_stack)
      smallest_stacks=[((i,j),stack) for i,bay in enumerate(new_module) for j,stack in enumerate(bay) if len(get_to_move([container.value for container in stack.containers],container_value))==min_M_S and not stack.is_full()and not (i,j)==source_stack]
      if smallest_stacks==[]:
        housekeepable_containers.remove(container_pos)
        continue
      else:
        destination_stack=random.choice(smallest_stacks)
        destination_stack_index=destination_stack[0]

    # 3. Move containers on top and in destination stack

    on_top=new_module[container_pos[0]][container_pos[1]][container_pos[2]+1:]
    destination_stack_to_move=list(get_to_move(new_module[destination_stack_index[0]][destination_stack_index[1]][:],container_value))

    while on_top or destination_stack_to_move:
      # If on_top is empty, give it value -1
      if len(on_top)==0:
        top_on_top=-1
      else:
        top_on_top=on_top[-1]

      # Same for destination_stack
      if len(destination_stack_to_move)==0:
        top_destination_stack_to_move=-1
      else:
        top_destination_stack_to_move=destination_stack_to_move[-1]

      # Pick the one with highest value to move first
      if top_on_top>=top_destination_stack_to_move:
        container_value=on_top.pop(-1)
        stack_index=source_stack
      else:
        container_value=destination_stack_to_move.pop(-1)
        stack_index=destination_stack_index
      # Get best stack to move this container to
      destination_stack=get_destination_stack(new_module,container_value,alpha,stack_index,excluded_stacks=[source_stack,destination_stack_index])
      print(f"Moving {stack_index} to {destination_stack}")
      new_module.move(stack_index,destination_stack)
      new_moves.append((stack_index,destination_stack))
      print(new_module)


    # 4. Move
    new_module.move(source_stack,destination_stack_index)
    new_moves.append((source_stack,destination_stack_index))
    print(f"Moving {source_stack} to {destination_stack_index}")
    print(new_module)

    # 5. Recalculate HC and SC:
    SC_new,_=CalculateShiftingCosts(new_module)

    # Calculate improvement
    improvement=sum(SC.values())-sum(SC_new.values())-len(new_moves)*alpha
    print(f"before: {sum(SC.values())}")
    print(f"after: {sum(SC_new.values())}")

    # If improvement, perform moves
    if improvement>0:
      print(f"Improvement: {improvement}")
      module=new_module
      moves.extend(new_moves)
      SC=SC_new
      HC,_=CalculateHousekeepingCosts(module,alpha)
      housekeepable_containers = [pos for pos in HC if HC[pos] < SC[pos] and HC[pos]>0]
    # If not, go to next container
    else:
      print(f"No improvement{improvement}")
      housekeepable_containers.remove(container_pos)
      continue
  return module,moves




In [20]:
print(module)

Bay 0:
  | 1 |   | 5
  | 4 |   | 4
2 | 3 |   | 4
1 | 3 | 2 | 1


In [29]:
new_module,moves=LookAheadHousekeeping(module,0.6)

housekeepable containers:[(0, 0, 1)]
Housekeeping:(0, 0, 1)
HC:0.8999999999999999
SC:1.25
Moving (0, 0) to (0, 2)
Bay 0:
  | 1 |   | 5
  | 4 |   | 4
  | 3 | 2 | 4
1 | 3 | 2 | 1
before: 7.25
after: 6.5
Improvement: 0.15000000000000002
housekeepable containers:[(0, 3, 3)]
Housekeeping:(0, 3, 3)
HC:1.2
SC:1.5
Ideal stacks: [((0, 2), Stack([Container(2, '', False), Container(2, '', False)]), 0.0)]
Moving (0, 0) to (0, 2)
Bay 0:
  | 1 |   | 5
  | 4 | 1 | 4
  | 3 | 2 | 4
  | 3 | 2 | 1
Moving (0, 3) to (0, 0)
Bay 0:
  | 1 |   |  
  | 4 | 1 | 4
  | 3 | 2 | 4
5 | 3 | 2 | 1
before: 6.5
after: 4.0
Improvement: 1.3
housekeepable containers:[(0, 3, 2)]
Housekeeping:(0, 3, 2)
HC:0.6
SC:1
Moving (0, 3) to (0, 0)
Bay 0:
  | 1 |   |  
  | 4 | 1 |  
4 | 3 | 2 | 4
5 | 3 | 2 | 1
before: 4.0
after: 3.3333333333333335
Improvement: 0.06666666666666654
housekeepable containers:[(0, 3, 1)]
Housekeeping:(0, 3, 1)
HC:0.8999999999999999
SC:1.3333333333333335
Moving (0, 3) to (0, 0)
Bay 0:
  | 1 |   |  
4 | 4 | 1 